# Collaboration and Review

We can use `rubicon` to share experiment results with teammates and collaborators by `publishing` experiments.

After we've run a few experiments in the *Iris Classifier* example, let's publishing those experiments for sharing with collaborators or reviewers.

In [ ]:
from rubicon import Rubicon

root_dir = "./rubicon-root"
rubicon = Rubicon(persistence="filesystem", root_dir=root_dir)

help(rubicon.publish)

In [ ]:
catalog_yml = rubicon.publish("Iris Model")
print(catalog_yml)

**Catalogs** are simply YAML files that the `rubicon` Intake driver can read and convert back to native `rubicon` objects. **The publishing process does not copy or move any data from the source it was originally logged to.** **Catalogs** point to the source data and tell the `rubicon` Intake driver how to read it. 

*Access controls should be properly defined on the underlying storage layer such that experimenters and reviewers can read and write as appropriate.*

___

In the above **catalog**, we published all of the *Iris Model* project's experiments. We can use the experiments' IDs to publish a subset of them. The next one only contains the *Iris Model* project's first experiment.

In [ ]:
project = rubicon.get_project("Iris Model")
first_experiment = project.experiments()[0]

catalog_yml = rubicon.publish("Iris Model", experiment_ids=[first_experiment.id])
print(catalog_yml)

We can also use the experiments' tags to select which ones we want to publish. Let's make a **catalog** with all our successful experiments.

In [ ]:
catalog_yml = rubicon.publish("Iris Model", experiment_tags=["success"])
print(catalog_yml)

At this point, sharing our published experiments is a copy/paste away! We can share the YAML with whoever we want to use it, and they can use `rubicon` to load the referenced experiments. If we'd rather send a file, we can write one by specifying the `output_filepath` parameter.

In [ ]:
rubicon.publish("Iris Model", experiment_tags=["success"], output_filepath="./catalog.yml")

! ls -l catalog.yml
! cat catalog.yml

___

Now that we've published experiments to a file ( `./catalog.yml` ), we could have collaborators or reviewers import them! Let's see how:

In [ ]:
import intake

catalog_path = "./catalog.yml"
catalog = intake.open_catalog(catalog_path)

list(catalog)

Above, we listed each of the **sources** within the catalog we loaded. Each **source** represents a project or experiment.

In [ ]:
source_name = [
    name for name in list(catalog) if "experiment" in name
][0]
source = catalog[source_name]

print(source)

We can call discover on a source to show its metadata, and read to load the full experiment into memory. Until we call one of those, Intake won't reach out to the underlying storage layer or load the source data. At no point will the source data be physically copied to the local filesystem. read returns native rubicon projects and experiments, so once we've got them we can do anything we normally could.

In [ ]:
from pprint import pprint
pprint(source.discover())

experiment = source.read()
print(experiment)

Let's see how this experiment was trained and how well it performed.

In [ ]:
for parameter in experiment.parameters():
    print(parameter.name, parameter.value)
print()

for feature in experiment.features():
    print(feature.name, feature.importance)
print()
    
for metric in experiment.metrics():
    print(metric.name, metric.value)

The catalog we've been given to review may only contain experiments the experimenter wanted to highlight, like ones with an accuracy above a certain threshold. We can still load all the experiments they logged to their project because catalogs generated by rubicon.publish contain a source for the parent project as well.

In [ ]:
project_source_name = [
    name for name in list(catalog) if "project" in name
][0]
source = catalog[project_source_name]

pprint(source.discover())

In [ ]:
project = source.read()
experiments = project.experiments()

print(len(experiments), "total experiments")
print(experiments)

Now we can dive into each experiment like we did the one above.